In [11]:
with open('2023_23.txt') as fh:
    input_data = fh.read()

# Splitting the input into lines
lines = input_data.split("\n")
grid = [list(line) for line in lines]
from collections import deque


# Directions for slope arrows
directions = {
    '^': (-1, 0),
    '>': (0, 1),
    'v': (1, 0),
    '<': (0, -1)
}

# Find the starting position 'S'
start_pos = None
for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        if cell == '.':
            start_pos = (i, j)
            break
    if start_pos:
        break

# BFS algorithm to find the longest path
def bfs_longest_path(start_pos, grid):
    # Queue for BFS, each element is (position, path, visited)
    queue = deque([(start_pos, [start_pos], set([start_pos]))])
    longest_path = []

    while queue:
        pos, path, visited = queue.popleft()

        # Check for path end condition (bottom row)
        if pos[0] == len(grid) - 1:
            if len(path) > len(longest_path):
                longest_path = path
            continue

        # Get possible next positions
        for d in directions.values():
            new_pos = (pos[0] + d[0], pos[1] + d[1])

            # Check if new position is valid and not visited
            if 0 <= new_pos[0] < len(grid) and 0 <= new_pos[1] < len(grid[0]) \
                    and new_pos not in visited and grid[new_pos[0]][new_pos[1]] != '#':
                # If current position is a slope, must follow direction
                if grid[pos[0]][pos[1]] in directions:
                    if directions[grid[pos[0]][pos[1]]] != d:
                        continue

                new_path = path + [new_pos]
                new_visited = visited.union({new_pos})
                queue.append((new_pos, new_path, new_visited))

    return longest_path

# Find the longest path
longest_path = bfs_longest_path(start_pos, grid)

# Mark the longest path on the grid with 'O' and starting position with 'S'
for i, j in longest_path:
    grid[i][j] = 'O'
grid[start_pos[0]][start_pos[1]] = 'S'

# Convert grid to string for display
longest_path_grid = [''.join(row) for row in grid]
longest_path_grid
print('Part 1:',len(longest_path)-1)

Part 1: 2366


In [12]:
with open('2023_23.txt') as fh:
    input_data = fh.read()

# Splitting the input into lines
lines = input_data.split("\n")
grid = [list(line) for line in lines]
from collections import deque

print
# Directions for slope arrows
directions = {
    '^': (-1, 0),
    '>': (0, 1),
    'v': (1, 0),
    '<': (0, -1)
}

# Find the starting position 'S'
start_pos = None
for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        if cell == '.':
            start_pos = (i, j)
            break
    if start_pos:
        break

def dfs_longest_path(start_pos, grid):
    # Stack for DFS, each element is (position, path, visited)
    stack = [(start_pos, [start_pos], set([start_pos]))]
    longest_path = []

    while stack:
        pos, path, visited = stack.pop()

        # Check for path end condition (bottom row)
        if pos[0] == len(grid) - 1:
            if len(path) > len(longest_path):
                longest_path = path
            continue

        # Get possible next positions
        for d in directions.values():
            new_pos = (pos[0] + d[0], pos[1] + d[1])

            # Check if new position is valid and not visited
            if 0 <= new_pos[0] < len(grid) and 0 <= new_pos[1] < len(grid[0]) \
                    and new_pos not in visited and grid[new_pos[0]][new_pos[1]] != '#':
                # If current position is a slope, must follow direction
                if grid[pos[0]][pos[1]] in directions:
                    if directions[grid[pos[0]][pos[1]]] != d:
                        continue

                new_path = path + [new_pos]
                new_visited = visited.union({new_pos})
                stack.append((new_pos, new_path, new_visited))

    return longest_path

# Find the longest path using DFS
longest_path_dfs = dfs_longest_path(start_pos, grid)

print('Part 1:',len(longest_path_dfs)-1)  # Subtract 1 because the start position is included in the path count



Part 1: 2366


In [13]:
with open('2023_23_2.txt') as fh:
    input_data = fh.read()

# Splitting the input into lines
lines = input_data.split("\n")
grid = [list(line) for line in lines]

def identify_junctions(grid):
    """Identify junctions in the grid."""
    rows, cols = len(grid), len(grid[0])
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # Up, Right, Down, Left
    junctions = [(0,1)]

    for r in range(rows):
        for c in range(cols):
            if grid[r][c] != '#':  # Only consider path cells
                viable_neighbors = 0
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] != '#':
                        viable_neighbors += 1

                if viable_neighbors > 2:
                    junctions.append((r, c))

    junctions.append((rows-1, cols-2))


    return junctions


# Identify junctions in the grid
junctions = identify_junctions(grid)
junctions[:10]  # 

def dfs_between_junctions(grid, start_junction, junctions):
    """Performs a DFS between junctions, terminating when another junction is reached."""
    stack = [(start_junction, [start_junction], set())]
    edges = {} # end_junc -> longest path

    while stack:
        pos, path, visited = stack.pop()
        visited.add(pos)

        # Check if we've reached another junction
        if pos in junctions and pos != start_junction:
            # print('junction found', pos)
            lpath = edges.get(pos, 0)
            if len(path) > lpath:
                edges[pos] = len(path)-1
            continue

        # Directions: Up, Right, Down, Left
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        for dr, dc in directions:
            next_pos = (pos[0] + dr, pos[1] + dc)
            if next_pos not in visited and 0 <= next_pos[0] < len(grid) and 0 <= next_pos[1] < len(grid[0]) and grid[next_pos[0]][next_pos[1]] != '#':
                stack.append((next_pos, path + [next_pos], visited.copy()))
        

    return edges


edges = [[0 for _ in range(len(junctions))] for _ in range(len(junctions))]
# Find the longest paths between each pair of junctions
for start_id, start_junc in enumerate(junctions):
    for end_junc, longest_path in dfs_between_junctions(grid, start_junc, set(junctions)).items():
        edges[junctions.index(start_junc)][junctions.index(end_junc)] = longest_path
        
def maximal_path_dfs(matrix, current_node, target_node, visited, current_length):
    if current_node == target_node:
        return current_length

    visited.add(current_node)
    max_length = 0

    for neighbor in range(len(matrix)):
        if matrix[current_node][neighbor] != 0 and neighbor not in visited:
            max_length = max(max_length, maximal_path_dfs(matrix, neighbor, target_node, visited, current_length + matrix[current_node][neighbor]))

    visited.remove(current_node)
    return max_length

def maximal_path(matrix):
    start_node = 0
    end_node = len(matrix) - 1
    visited = set()

    return maximal_path_dfs(matrix, start_node, end_node, visited, 0)


print('Part 2:',maximal_path(edges))  # Replace with your matrix        


Part 2: 6682
